In [ ]:
#====================================================================
#PROGRAM INFORMATION
#====================================================================
#PROGRAM NAME: Doggo_WGAN-GP
#AUTHOR(S):         David Helminiak, EECE, Marquette University
#
#DATE CREATED:      April 2020
#DATE MODIFIED:     May 2020
#VERSION NUM:       1.0
#DESCRIPTION:       Uses an optimized WGAN-GP model to generate
#                   64x64 color images of dogs. 
#====================================================================

import cv2 
import glob
import tensorflow as tf
import matplotlib.pyplot as plt
import natsort
import numpy as np
import os
import random
import sys
import shutil
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *
from PIL import Image
from IPython.display import clear_output
from IPython.core.debugger import Tracer

#Establish GPU distribution strategy
strategy = tf.distribute.MirroredStrategy()


In [ ]:
def makeGenerator():
    
    #Create a sequential NN model
    model = Sequential()
    
    #8x8x512
    model.add(Dense(8*8*512, input_shape=(randomNoiseSize,)))
    model.add(LeakyReLU())
    model.add(Reshape((8,8,512)))

    #16x16x512
    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(512, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())
    model.add(Conv2D(512, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())
    
    #32x32x256
    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(256, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())
    model.add(Conv2D(256, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())
    
    #64x64x128
    model.add(UpSampling2D(interpolation='nearest'))
    model.add(Conv2D(128, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())
    model.add(Conv2D(128, (winSize, winSize), strides=(1,1), kernel_initializer='he_normal', padding='same'))
    model.add(LayerNormalization(epsilon=epsilonValue))
    model.add(LeakyReLU())

    #Activate
    model.add(Conv2D(3, (winSize,winSize), strides=(1,1), kernel_initializer='he_normal', padding='same', activation='tanh'))
    
    return model
    
    
def makeDiscriminator():
    model = Sequential()
    
    #32x32x64  
    model.add(Conv2D(64, (winSize,winSize), strides=(2,2), padding='same', input_shape=[outputSize, outputSize, numChannels], kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    model.add(Conv2D(64, (winSize,winSize), strides=(1,1), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    
    #16x16x128
    model.add(Conv2D(128, (winSize,winSize), strides=(2,2), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    model.add(Conv2D(128, (winSize,winSize), strides=(1,1), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())

    #8x8x256
    model.add(Conv2D(256, (winSize,winSize), strides=(2,2), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    model.add(Conv2D(256, (winSize,winSize), strides=(1,1), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    
    #4x4x512
    model.add(Conv2D(512, (winSize,winSize), strides=(2,2), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    model.add(Conv2D(512, (winSize,winSize), strides=(1,1), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU())
    
    #Activate
    model.add(Flatten())
    model.add(Dense(1, kernel_initializer='he_normal'))
    return model


In [ ]:

#Define a training step
def trainStep(images, trainGenFlag):

    #Create some random noise to generate some fake images
    noise = tf.random.normal([batchSize, randomNoiseSize], dtype=tf.float32)
    
    with tf.GradientTape() as genTape, tf.GradientTape() as disTape:

        #Generate fake images with the generator
        fakes = generator(noise, training=True)

        #Add some noise to the real images
        #realInputWithNoise = images + tf.random.normal(shape=tf.shape(images), mean=0.0, stddev=random.uniform(0.0, 0.1), dtype=tf.float32)

        #Determine the discriminator results for the fake images
        disFake = discriminator(fakes, training=True)

        #Determine the discriminator results for the real images
        disReal = discriminator(images, training=True)

        #Determine the discriminator loss
        disLoss = tf.reduce_mean(disFake) - tf.reduce_mean(disReal)

        #Calculate and add the gradient penalty
        alpha = tf.random.uniform([batchSize, 1, 1, 1], 0., 1.)
        interpImage = images + (alpha * (fakes - images))
        disInterp = discriminator(interpImage, training=True)
        grad = tf.gradients(disInterp, [interpImage])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(grad), axis=[1, 2, 3])+0.0)
        gp = tf.reduce_mean((slopes - 1.)**2)
        disLoss += gradientWeightPenalty * gp

        #Caclulate the discriminator gradients
        disGrad = disTape.gradient(disLoss, discriminator.trainable_variables)

        #Use the gradients to optimize the discriminator
        disOptimizer.apply_gradients(zip(disGrad, discriminator.trainable_variables))

        #If this step the generator should also be trained
        if trainGenFlag: 

            #Determine the generator loss
            genLoss = -1. * tf.reduce_mean(disFake)

            #Caclulate the generator gradients
            genGrad = genTape.gradient(genLoss, generator.trainable_variables)

            #Use the gradients to optimize the generator
            genOptimizer.apply_gradients(zip(genGrad, generator.trainable_variables))

            return [genLoss, disLoss]
        else:
            return disLoss

#Define execution of training step given the system computational scope
@tf.function
def distributed_train_step(images, trainGenFlag):
    if trainGenFlag:
        genLosses, disLosses = strategy.run(trainStep, args=(images, trainGenFlag,))
        return [strategy.reduce(tf.distribute.ReduceOp.MEAN, genLosses, axis=None), 
                strategy.reduce(tf.distribute.ReduceOp.MEAN, disLosses, axis=None)]
    else:
        disLosses = strategy.run(trainStep, args=(images, trainGenFlag,))
        return strategy.reduce(tf.distribute.ReduceOp.MEAN, disLosses, axis=None)

#Define the training procedure
def train(dataset, numEpochs):
    results = []
    bestDisLoss = np.nan
    bestGenerator = np.nan
    bestDiscriminator = np.nan
    epochsSinceImproved = 0
    for epoch in range(numEpochs):
        
        batchDisResults = []
        batchGenResults = []
        trainIter = trainGenRate-1
        for batch in dataset:
            trainIter+=1
            if trainIter >= trainGenRate:
                result = distributed_train_step(batch, True) 
                batchGenResults.append(result[0].numpy())
                batchDisResults.append(result[1].numpy())
                trainIter = 0
            else:
                result = distributed_train_step(batch, False) 
                batchDisResults.append(result.numpy())
            
        results.append([np.mean(batchGenResults), np.mean(batchDisResults)])
        
        #Wipe the screen of prior outputs
        clear_output()
        
        #Generate fake images using consistent randomly chosen seeds
        fakeImages = tf.reshape(generator(seed, training=False), shape=(4, outputSize,outputSize,numChannels))
        
        #Plot losses and fake images
        f = plt.figure(figsize=(20,10))
        f.subplots_adjust(top = 0.85)
        f.subplots_adjust(wspace=0.2, hspace=0.2)
        plt.suptitle('Epoch:'+str(epoch), fontsize=20, fontweight='bold', y = 0.95)

        ax1 = plt.subplot2grid((2,4), (0,0), colspan=2)
        ax1.plot(np.asarray(results)[:,0])
        ax1.set_title('Generator Loss: ' + str(np.mean(batchGenResults)), fontsize=15, fontweight='bold')

        ax2 = plt.subplot2grid((2,4), (1,0), colspan=2)
        ax2.plot(np.asarray(results)[:,1])
        ax2.set_title('Discriminator Loss: '+ str(np.mean(batchDisResults)), fontsize=15, fontweight='bold')
        
        ax3 = plt.subplot2grid((2,4), (0,2))
        ax3.imshow(((fakeImages[0] * 127.5 + 127.5) / 255.).numpy())

        ax4 = plt.subplot2grid((2,4), (0,3))
        ax4.imshow(((fakeImages[1] * 127.5 + 127.5) / 255.).numpy())

        ax5 = plt.subplot2grid((2,4), (1,2))
        ax5.imshow(((fakeImages[2] * 127.5 + 127.5) / 255.).numpy())

        ax6 = plt.subplot2grid((2,4), (1,3))
        ax6.imshow(((fakeImages[3] * 127.5 + 127.5) / 255.).numpy())
        plt.show()
        
        #Check epoch number, save loss/fake image information
        if (epoch%1) == 0: f.savefig(dir_output + '/' + str(epoch) + '_epoches.png', bbox_inches='tight')
        
        #Check epoch number
        if (epoch%50) == 0:
            #Save model checkpoint
            dir_checkpointSave = dir_checkpoint+'/checkpoint-'+str(epoch)+'/'
            os.makedirs(dir_checkpointSave)
            checkpoint.save(file_prefix = dir_checkpointSave)



In [ ]:
#USER PARAMETERS
#===========================================================

#Should the modelbe trained, or loaded from prior run
trainModel = True

#If loading a prior run, which epoch should be loaded; runs that don't exist will not restore a model
restoreEpochNum = -1

#Define the initial learning rate for the optimizer
disLearningRate=1e-4
genLearningRate=1e-4

#How often (batch iterations) should the generator be trained
trainGenRate = 5

#Define the number of samples fed through each step
batchSize = 8
gBatchSize = batchSize * strategy.num_replicas_in_sync

#How many epochs should be run at minimum
numEpochs = 10000

#Specify the output/input image size
outputSize = 2**6

#Specify the number of output/input channels
numChannels = 3

#Specify the convolution window size; #columns, #rows
winSize = 3

#Specify the stride length
strideSize = 2

#Number of convolutional layers for the generator and discriminator
numConv = 4

#Length of the random noise vector; made to correspond with max number of channels in the generator
randomNoiseSize = 128

epsilonValue = 0.00001
gradientWeightPenalty = 10

#STATIC PARAMETERS
#===========================================================
#Output directory
dir_output = './Results'

#Output directory for final generated images
dir_genOutput = './Generated'

#Checkpoint directory
dir_checkpoint = './Checkpoints'

#Training data directory
#dir_trainData = './Training_Anime/'
#dir_trainData = './Training_Pokemon/'
dir_trainData = './Training_Doggos/'

#Testing data directory
#dir_testData = './Testing_Data/'

#Clear output and checkpoint directories
if os.path.exists(dir_output): shutil.rmtree(dir_output)
os.makedirs(dir_output)

if os.path.exists(dir_genOutput): shutil.rmtree(dir_genOutput)
os.makedirs(dir_genOutput)

if os.path.exists(dir_checkpoint) and trainModel: 
    shutil.rmtree(dir_checkpoint)
    os.makedirs(dir_checkpoint)



In [ ]:

#MAIN CODE EXECUTION
#===========================================================

#print('Loading Data')
#(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
#train_images = np.asarray([cv2.resize(img,(int(outputSize),int(outputSize))) for img in train_images])
#train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
#train_images = (train_images - 127.5) / 127.5
#trainData = tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(batchSize)
if trainModel:
    
    #Inform user
    print('Loading Training Data')

    #Load in training data
    #trainImages = np.asarray([cv2.resize(cv2.imread(imageFileName, -1),(int(outputSize),int(outputSize))) for imageFileName in glob.glob(dir_trainData + '/*.png')])
    trainImages = np.asarray([np.asarray(Image.open(imageFileName).convert('RGB').resize((int(outputSize),int(outputSize)))) for imageFileName in glob.glob(dir_trainData + '/*.png')])
    #Save the number of training images
    numTraining = len(trainImages)

    #Images should be normalized to be between -1 and 1 for visualization and match with noise input
    trainImages = (trainImages-127.5)/127.5
    trainImages = trainImages.astype('float32')

    #Setup tensors from the training data for input into the network
    trainImages = trainImages.reshape(trainImages.shape[0], trainImages.shape[1], trainImages.shape[2], trainImages.shape[3])
    trainData = tf.data.Dataset.from_tensor_slices(trainImages)

    #Shuffle and batch the training data
    trainData = trainData.shuffle(numTraining).batch(gBatchSize, drop_remainder=True)
    
    #Prepare the data for distribution across multiple GPUs
    trainData = strategy.experimental_distribute_dataset(trainData)

#Define cross entropy metrics
#0.2: 0.1 for False and 0.9 for True
#0.3: 0.2 0.8
#0.4: 0.3 0.7
#label_smoothing=0.1
#disc_crossEntropy = keras.losses.BinaryCrossentropy(from_logits = False)
#gen_crossEntropy = keras.losses.BinaryCrossentropy(from_logits = False)

#Spcify the optimizers for the generator and discriminator networks
#genOptimizer = RMSprop(learning_rate=genLearningRate, rho=0.9)
#disOptimizer = RMSprop(learning_rate=disLearningRate, rho=0.9)

#genOptimizer = Adam(learning_rate=genLearningRate, beta_1=0.5)
#disOptimizer = Adam(learning_rate=disLearningRate, beta_1=0.5)
with strategy.scope():
    genOptimizer = Nadam(learning_rate=genLearningRate, beta_1=0.5, beta_2=0.9)
    disOptimizer = Nadam(learning_rate=disLearningRate, beta_1=0.5, beta_2=0.9)

#Create a random seed for noise generation
seed = tf.random.normal([4, randomNoiseSize], dtype=tf.float32)

#Create a discriminator
print("Creating a discriminator")
with strategy.scope(): discriminator = makeDiscriminator()

#Create a generator
print("Creating a generator")
with strategy.scope(): generator = makeGenerator()

#Create a checkpoint object
with strategy.scope(): checkpoint = tf.train.Checkpoint(generator_optimizer=genOptimizer, discriminator_optimizer=disOptimizer, generator=generator, discriminator=discriminator)


In [ ]:
#Print out generator structure
generator.summary()

In [ ]:
#Print out discriminator structure
discriminator.summary()

In [ ]:

#If training the model from scratch
if trainModel:
    
    #Inform user
    print('Building and training model')
    
    #Use GPU to perform training
    errCode = train(trainData, numEpochs)

#If just loading a prior model
else: 
    
    #Inform user
    print('Loading prior model')
    
    #Restore specfied checkpoint
    checkpoint.restore(tf.train.latest_checkpoint(dir_checkpoint+'/checkpoint-'+str(restoreEpochNum)+'/'))

    

In [ ]:
#Randomly generate a a bunch of doggos, use the discriminator to verify that they are indeed dogs

restoreEpochNum = 5000
checkpoint.restore(tf.train.latest_checkpoint(dir_checkpoint+'/checkpoint-'+str(restoreEpochNum)+'/'))
checkpoint.restore(tf.train.latest_checkpoint(dir_checkpoint))

#Inform user
print('Generating Some New Images')

#Loop until completion
imSaved = 0
while True:
    
    #Create a new fake image
    truncation = 1  # Truncation trick, can trade fidelity for variation [0-low variation, 1-max variation]
    seed = truncation * tf.random.truncated_normal(shape = [1, randomNoiseSize], dtype=tf.float32)
    fakeImage = tf.reshape(generator(seed, training=False), shape=(outputSize,outputSize,numChannels))
    
    #Query the discriminator
    discrimResult = discriminator(fakeImage.numpy().reshape(1, outputSize, outputSize, numChannels), training=False).numpy()[0][0]
    
    #If the discriminator was sufficiently fooled, save the image
    imSaved+=1
    fakeImage = ((fakeImage * 127.5 + 127.5) / 255.).numpy()
    fakeImage = (255.0 / np.max(fakeImage) * (fakeImage - np.min(fakeImage))).astype(np.uint8)

    im = Image.fromarray(fakeImage)
    im.save(dir_genOutput + '/' + 'genImage_'+str(imSaved)+'_score'+str(discrimResult)+'.png')
    if imSaved == 200: break

### 